In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from PIL import Image
import random
import warnings
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy, Accuracy
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import ResNet50, MobileNetV2, DenseNet169
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Checking GPU availability

print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Loading train and test dataframes

train_df = pd.read_csv('./../data/cleaned/train.csv')
test_df = pd.read_csv('./../data/cleaned/test.csv')

In [ ]:
train_dir = './../data/cleaned/ferplus_affectnet/train/'
test_dir = './../data/cleaned/ferplus_affectnet/test/'

input_size = (48, 48)
labels = ['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

train_datagen = ImageDataGenerator(rescale=1./255,
                                   validation_split=0.2,
                                   rotation_range=0.3,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=False,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=(48, 48),
                                                    class_mode='categorical',
                                                    subset='training',
                                                    classes=labels,
                                                    batch_size=32)

val_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                target_size=(48, 48),
                                                class_mode='categorical',
                                                subset='validation',
                                                classes=labels,
                                                batch_size=32)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(directory=test_dir,
                                                  target_size=(48, 48),
                                                  class_mode='categorical',
                                                  classes=labels,
                                                  batch_size=32)

In [ ]:
model = Sequential()

# Conv layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten and dense layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(8, activation='softmax'))

# Compiling the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Fitting model

model_path = './../models/best_custom_model.h5'

callbacks = [EarlyStopping(patience=10, monitor='val_loss', mode='min'), 
             ReduceLROnPlateau(patience=2, verbose=1),
             ModelCheckpoint(model_path, save_best_only=True, monitor='val_accuracy')]

history = model.fit(train_generator, validation_data=val_generator, epochs=25, batch_size=64, verbose=2, callbacks=callbacks)

In [ ]:
# Plotting train and val losses

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Model evaluation on val set

val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

In [ ]:
# Making predictions on test set

y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

y_true = test_generator.classes

In [ ]:
y_true

In [ ]:
# Performance metrics on test set

cr = classification_report(y_pred_labels, y_true)
print(f'Classification report:\n{cr}')

chs = cohen_kappa_score(y_pred_labels, y_true)
print(f'Cohen-Kappa score: {chs}')

cm = confusion_matrix(y_pred_labels, y_true)

plt.figure(figsize=(8, 6))
sns.set(font_scale=0.9)
sns.heatmap(cm, annot=True, cmap='Blues', xticklabels=labels, yticklabels=labels, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.xticks(rotation=90)
plt.show()